In [1]:
import os
import uproot
import pandas as pd
from tqdm import tqdm
import pandas as pd
from multiprocessing import Pool, cpu_count, Manager, Lock
from tqdm.contrib.concurrent import process_map

In [2]:
# Function to check file integrity
def check_file_integrity(file_path, required_tree):
    try:
        with uproot.open(file_path) as file:
            if required_tree in file:
                tree = file[required_tree]

                # Check for empty tree
                if tree.num_entries == 0:
                    return "Tree has no entries"

                # Attempt to read data from each branch
                for branch in tree.keys():
                    try:
                        data = tree[branch].array()  # This might throw an error if data is corrupted
                        # Perform any additional checks on the data
                    except Exception as e:
                        return f"Error reading branch {branch}: {str(e)}"
            else:
                return "Required tree not found"
    except Exception as e:
        return f"File read error: {str(e)}"

    return None  # No issues found


In [3]:
# Function to check the keys in a file
def check_file_keys(file_path, required_tree, missing_tree_files):
    try:
        with uproot.open(file_path) as file:
            if required_tree not in file:
                missing_tree_files.append(os.path.basename(file_path))
                return None  # Tree not found, skip key check
            keys = file.keys()
            return keys
    except Exception as e:
        return str(e)

In [4]:
def process_file(args):
    file_path, required_tree, missing_tree_files = args
    key_check_result = check_file_keys(file_path, required_tree, missing_tree_files)
    integrity_result = check_file_integrity(file_path, required_tree)
    errors = []
    if isinstance(key_check_result, str):
        errors.append({'file': os.path.basename(file_path), 'error': key_check_result})
    elif isinstance(key_check_result, list):
        for key in key_check_result:
            try:
                with uproot.open(file_path) as file:
                    file[key]
            except Exception as e:
                errors.append({'file': os.path.basename(file_path), 'key': key, 'error': str(e)})

    if integrity_result:
        errors.append({'file': os.path.basename(file_path), 'error': integrity_result})

    return errors

In [5]:
def print_summary(error_files, missing_tree_files, error_summary, required_tree):
    print("Summary of Errors:")
    print(error_summary)

    print("\nNumber of files with errors:", len(error_files))
    if error_files:
        print("List of files with errors:\n" + "\n".join(error_files))

    print("\nNumber of files missing the tree '{}': {}".format(required_tree, len(missing_tree_files)))
    if missing_tree_files:
        print("List of files missing the tree:\n" + "\n".join(missing_tree_files))

In [8]:
import os
import pandas as pd
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

def check_files(data_folder_path, decay_trees, year=None, magnet=None, ll_or_dd=None):
    years = [year] if year else ['2016', '2017']
    magnets = [magnet] if magnet else ['MagDown', 'MagUp']
    ll_or_dds = [ll_or_dd] if ll_or_dd else ['LL', 'DD']
    config_errors = {}  # Dictionary to hold error details per configuration

    with Pool(processes=cpu_count()) as pool:
        for decay_tree_base in decay_trees:
            for year in years:
                for magnet in magnets:
                    for ll_or_dd in ll_or_dds:
                        config_key = (decay_tree_base, year, magnet, ll_or_dd)
                        config_errors[config_key] = {'error_files': [], 'error_details': [], 'missing_tree_files': []}

                        output_folder = os.path.join(data_folder_path, decay_tree_base, year, magnet, ll_or_dd)
                        output_files = [f for f in os.listdir(output_folder) if f.endswith('.root')]
                        required_tree = f"{decay_tree_base}_{ll_or_dd}/DecayTree"

                        tasks = [(os.path.join(output_folder, filename), required_tree, config_errors[config_key]['missing_tree_files']) for filename in output_files]
                        chunksize = max(1, len(tasks) // (10 * cpu_count()))

                        with tqdm(total=len(tasks), desc=f"Processing {config_key}") as pbar:
                            for result in pool.imap(process_file, tasks, chunksize=chunksize):
                                if isinstance(result, str):
                                    config_errors[config_key]['error_files'].append(result)
                                    config_errors[config_key]['error_details'].append({'file': result, 'error': result})
                                elif result is None:
                                    continue
                                elif isinstance(result, list):
                                    for error in result:
                                        config_errors[config_key]['error_files'].append(error['file'])
                                        config_errors[config_key]['error_details'].append(error)
                                pbar.update(1)

    # Display statistics for each configuration
    for config, data in config_errors.items():
        print(f"\nConfiguration: {config}")

        # Create DataFrame from list of dictionaries
        error_summary = pd.DataFrame(data['error_details'])

        # Apply table styles
        error_summary_styled = error_summary.style.set_table_styles([
            {'selector': '', 'props': [('border', '1px solid black')]}
        ])

        # Print the DataFrame
        print("Summary of Errors:")
        display(error_summary_styled)

        print("\nNumber of files with errors:", len(data['error_files']))
        if data['error_files']:
            print("List of files with errors:\n" + "\n".join(data['error_files']))

        print("\nNumber of files missing the tree '{}': {}".format(required_tree, len(data['missing_tree_files'])))
        if data['missing_tree_files']:
            print("List of files missing the tree:\n" + "\n".join(data['missing_tree_files']))


## Run the checks

### B2L0barPKpKm

In [10]:

# usage
data_folder_path = "/eos/lhcb/user/m/melashri/data/red_RD"
decay_trees = ['B2L0barPKpKm']
check_files(data_folder_path, decay_trees)

Processing ('B2L0barPKpKm', '2016', 'MagDown', 'LL'): 100%|██████████| 1625/1625 [03:19<00:00,  8.16it/s]
Processing ('B2L0barPKpKm', '2016', 'MagDown', 'DD'): 100%|██████████| 1625/1625 [03:16<00:00,  8.28it/s]
Processing ('B2L0barPKpKm', '2016', 'MagUp', 'LL'): 100%|██████████| 1454/1454 [02:56<00:00,  8.26it/s]
Processing ('B2L0barPKpKm', '2016', 'MagUp', 'DD'): 100%|██████████| 1454/1454 [02:55<00:00,  8.29it/s]
Processing ('B2L0barPKpKm', '2017', 'MagDown', 'LL'): 100%|██████████| 1434/1434 [02:53<00:00,  8.25it/s]
Processing ('B2L0barPKpKm', '2017', 'MagDown', 'DD'): 100%|██████████| 1434/1434 [02:53<00:00,  8.27it/s]
Processing ('B2L0barPKpKm', '2017', 'MagUp', 'LL'): 100%|██████████| 1364/1364 [02:45<00:00,  8.22it/s]
Processing ('B2L0barPKpKm', '2017', 'MagUp', 'DD'): 100%|██████████| 1364/1364 [02:44<00:00,  8.30it/s]



Configuration: ('B2L0barPKpKm', '2016', 'MagDown', 'LL')
Summary of Errors:


,file,error
0,datadata_2016MagDown_195_reduced.root,Error reading branch p_ProbNNghost: index -1 is out of bounds for axis 0 with size 0
1,datadata_2016MagDown_226_reduced.root,Error reading branch L0_PE: index -1 is out of bounds for axis 0 with size 0
2,datadata_2016MagDown_438_reduced.root,Error reading branch Lpi_PZ: index -1 is out of bounds for axis 0 with size 0



Number of files with errors: 3
List of files with errors:
datadata_2016MagDown_195_reduced.root
datadata_2016MagDown_226_reduced.root
datadata_2016MagDown_438_reduced.root

Number of files missing the tree 'B2L0barPKpKm_DD/DecayTree': 0

Configuration: ('B2L0barPKpKm', '2016', 'MagDown', 'DD')
Summary of Errors:


,file,error
0,datadata_2016MagDown_1590_reduced.root,Error reading branch h1_P: index -1 is out of bounds for axis 0 with size 0
1,datadata_2016MagDown_392_reduced.root,Error reading branch Lp_PE: entries in normal baskets (0) plus embedded baskets (0) don't add up to expected number of entries (172) in file /eos/lhcb/user/m/melashri/data/red_RD/B2L0barPKpKm/2016/MagDown/DD/datadata_2016MagDown_392_reduced.root
2,datadata_2016MagDown_919_reduced.root,Error reading branch h2_PX: index -1 is out of bounds for axis 0 with size 0


...


Number of files with errors: 10
List of files with errors:
datadata_2017MagDown_513_reduced.root
datadata_2017MagDown_518_reduced.root
datadata_2017MagDown_643_reduced.root
datadata_2017MagDown_659_reduced.root
datadata_2017MagDown_683_reduced.root
datadata_2017MagDown_686_reduced.root
datadata_2017MagDown_882_reduced.root
datadata_2017MagDown_922_reduced.root
datadata_2017MagDown_931_reduced.root
datadata_2017MagDown_963_reduced.root

Number of files missing the tree 'B2L0barPKpKm_DD/DecayTree': 0

Configuration: ('B2L0barPKpKm', '2017', 'MagUp', 'LL')
Summary of Errors:


,file,error
0,datadata_2017MagUp_1004_reduced.root,Error reading branch h1_PE: index -1 is out of bounds for axis 0 with size 0
1,datadata_2017MagUp_1109_reduced.root,Error reading branch L0_ENDVERTEX_Y: index -1 is out of bounds for axis 0 with size 0
2,datadata_2017MagUp_1144_reduced.root,Error reading branch h1_PX: index -1 is out of bounds for axis 0 with size 0
3,datadata_2017MagUp_123_reduced.root,Error reading branch L0_PZ: entries in normal baskets (0) plus embedded baskets (0) don't add up to expected number of entries (185) in file /eos/lhcb/user/m/melashri/data/red_RD/B2L0barPKpKm/2017/MagUp/LL/datadata_2017MagUp_123_reduced.root
4,datadata_2017MagUp_1289_reduced.root,Error reading branch nBu_DTFL0_Lambda0_ctauErr: index -1 is out of bounds for axis 0 with size 0
5,datadata_2017MagUp_1306_reduced.root,Error reading branch Bu_Hlt1TrackMVADecision_TOS: index -1 is out of bounds for axis 0 with size 0
6,datadata_2017MagUp_1320_reduced.root,Error reading branch Lp_TRACK_Type: index -1 is out of bounds for axis 0 with size 0



Number of files with errors: 7
List of files with errors:
datadata_2017MagUp_1004_reduced.root
datadata_2017MagUp_1109_reduced.root
datadata_2017MagUp_1144_reduced.root
datadata_2017MagUp_123_reduced.root
datadata_2017MagUp_1289_reduced.root
datadata_2017MagUp_1306_reduced.root
datadata_2017MagUp_1320_reduced.root

Number of files missing the tree 'B2L0barPKpKm_DD/DecayTree': 0

Configuration: ('B2L0barPKpKm', '2017', 'MagUp', 'DD')
Summary of Errors:


,file,error
0,datadata_2017MagUp_1095_reduced.root,Error reading branch h1_PY: entries in normal baskets (0) plus embedded baskets (0) don't add up to expected number of entries (144) in file /eos/lhcb/user/m/melashri/data/red_RD/B2L0barPKpKm/2017/MagUp/DD/datadata_2017MagUp_1095_reduced.root
1,datadata_2017MagUp_119_reduced.root,Error reading branch nBu_DTF_decayLengthErr: index -1 is out of bounds for axis 0 with size 0
2,datadata_2017MagUp_345_reduced.root,Error reading branch Lp_PY: index -1 is out of bounds for axis 0 with size 0
3,datadata_2017MagUp_394_reduced.root,Error reading branch h1_PX: entries in normal baskets (0) plus embedded baskets (0) don't add up to expected number of entries (220) in file /eos/lhcb/user/m/melashri/data/red_RD/B2L0barPKpKm/2017/MagUp/DD/datadata_2017MagUp_394_reduced.root
4,datadata_2017MagUp_448_reduced.root,Error reading branch L0_P: index -1 is out of bounds for axis 0 with size 0
5,datadata_2017MagUp_495_reduced.root,Error reading branch Bu_PE: index -1 is out of bounds for axis 0 with size 0
6,datadata_2017MagUp_630_reduced.root,Error reading branch h1_P: index -1 is out of bounds for axis 0 with size 0
7,datadata_2017MagUp_713_reduced.root,Error reading branch Lp_PX: index -1 is out of bounds for axis 0 with size 0
8,datadata_2017MagUp_715_reduced.root,Error reading branch h2_PZ: index -1 is out of bounds for axis 0 with size 0
9,datadata_2017MagUp_771_reduced.root,Error reading branch L0_OWNPV_ZERR: index -1 is out of bounds for axis 0 with size 0



Number of files with errors: 10
List of files with errors:
datadata_2017MagUp_1095_reduced.root
datadata_2017MagUp_119_reduced.root
datadata_2017MagUp_345_reduced.root
datadata_2017MagUp_394_reduced.root
datadata_2017MagUp_448_reduced.root
datadata_2017MagUp_495_reduced.root
datadata_2017MagUp_630_reduced.root
datadata_2017MagUp_713_reduced.root
datadata_2017MagUp_715_reduced.root
datadata_2017MagUp_771_reduced.root

Number of files missing the tree 'B2L0barPKpKm_DD/DecayTree': 0


### B2L0barPKpKp

In [9]:

# usage
data_folder_path = "/eos/lhcb/user/m/melashri/data/red_RD"
decay_trees = ['B2L0PbarKpKp']
check_files(data_folder_path, decay_trees)

Processing ('B2L0PbarKpKp', '2016', 'MagDown', 'LL'): 100%|██████████| 1625/1625 [03:15<00:00,  8.31it/s]
Processing ('B2L0PbarKpKp', '2016', 'MagDown', 'DD'): 100%|██████████| 1625/1625 [03:13<00:00,  8.40it/s]
Processing ('B2L0PbarKpKp', '2016', 'MagUp', 'LL'): 100%|██████████| 1454/1454 [02:51<00:00,  8.46it/s]
Processing ('B2L0PbarKpKp', '2016', 'MagUp', 'DD'): 100%|██████████| 1454/1454 [02:52<00:00,  8.44it/s]
Processing ('B2L0PbarKpKp', '2017', 'MagDown', 'LL'): 100%|██████████| 1434/1434 [02:50<00:00,  8.43it/s]
Processing ('B2L0PbarKpKp', '2017', 'MagDown', 'DD'): 100%|██████████| 1434/1434 [02:50<00:00,  8.42it/s]
Processing ('B2L0PbarKpKp', '2017', 'MagUp', 'LL'): 100%|██████████| 1364/1364 [02:42<00:00,  8.41it/s]
Processing ('B2L0PbarKpKp', '2017', 'MagUp', 'DD'): 100%|██████████| 1364/1364 [02:41<00:00,  8.44it/s]



Configuration: ('B2L0PbarKpKp', '2016', 'MagDown', 'LL')
Summary of Errors:


,file,error
0,datadata_2016MagDown_1253_reduced.root,Error reading branch p_PY: index -1 is out of bounds for axis 0 with size 0
1,datadata_2016MagDown_1339_reduced.root,Error reading branch h1_IPCHI2_OWNPV: index -1 is out of bounds for axis 0 with size 0
2,datadata_2016MagDown_1588_reduced.root,Required tree not found
3,datadata_2016MagDown_592_reduced.root,Error reading branch Bu_DTF_ctau: index -1 is out of bounds for axis 0 with size 0
4,datadata_2016MagDown_602_reduced.root,Error reading branch h2_PY: index -1 is out of bounds for axis 0 with size 0
5,datadata_2016MagDown_721_reduced.root,Error reading branch h1_PY: index -1 is out of bounds for axis 0 with size 0
6,datadata_2016MagDown_755_reduced.root,Error reading branch h1_PIDK: index -1 is out of bounds for axis 0 with size 0
7,datadata_2016MagDown_840_reduced.root,Error reading branch h2_PY: index -1 is out of bounds for axis 0 with size 0
8,datadata_2016MagDown_889_reduced.root,Error reading branch p_PE: index -1 is out of bounds for axis 0 with size 0
9,datadata_2016MagDown_900_reduced.root,Error reading branch h1_PX: index -1 is out of bounds for axis 0 with size 0



Number of files with errors: 11
List of files with errors:
datadata_2016MagDown_1253_reduced.root
datadata_2016MagDown_1339_reduced.root
datadata_2016MagDown_1588_reduced.root
datadata_2016MagDown_592_reduced.root
datadata_2016MagDown_602_reduced.root
datadata_2016MagDown_721_reduced.root
datadata_2016MagDown_755_reduced.root
datadata_2016MagDown_840_reduced.root
datadata_2016MagDown_889_reduced.root
datadata_2016MagDown_900_reduced.root
datadata_2016MagDown_907_reduced.root

Number of files missing the tree 'B2L0PbarKpKp_DD/DecayTree': 0

Configuration: ('B2L0PbarKpKp', '2016', 'MagDown', 'DD')
Summary of Errors:


,file,error,key
0,datadata_2016MagDown_1190_reduced.root,Error reading branch Bu_DTFL0_status: index -1 is out of bounds for axis 0 with size 0,nan
1,datadata_2016MagDown_1195_reduced.root,Error reading branch h1_PIDK: index -1 is out of bounds for axis 0 with size 0,nan
2,datadata_2016MagDown_1210_reduced.root,Error reading branch L0_PE: index -1 is out of bounds for axis 0 with size 0,nan
3,datadata_2016MagDown_1255_reduced.root,Error reading branch p_PX: index -1 is out of bounds for axis 0 with size 0,nan
4,datadata_2016MagDown_1464_reduced.root,"while reading TTree version 13366 as uproot.dynamic.Model_TTree_v20 (929980213 bytes) (base): (base): TAttFill version 2 as uproot.dynamic.Model_TAttFill_v2 (12 bytes) fFillColor: 602 fFillStyle: 1 Members for TAttFill: (fFillColor), (fFillStyle) expected 12 bytes but cursor moved by 10 bytes (through TAttFill) in file /eos/lhcb/user/m/melashri/data/red_RD/B2L0PbarKpKp/2016/MagDown/DD/datadata_2016MagDown_1464_reduced.root in object /B2L0PbarKpKp_DD/DecayTree;1",B2L0PbarKpKp_DD/DecayTree;1
5,datadata_2016MagDown_1464_reduced.root,"File read error: while reading TTree version 13366 as uproot.dynamic.Model_TTree_v20 (929980213 bytes) (base): (base): TAttFill version 2 as uproot.dynamic.Model_TAttFill_v2 (12 bytes) fFillColor: 602 fFillStyle: 1 Members for TAttFill: (fFillColor), (fFillStyle) expected 12 bytes but cursor moved by 10 bytes (through TAttFill) in file /eos/lhcb/user/m/melashri/data/red_RD/B2L0PbarKpKp/2016/MagDown/DD/datadata_2016MagDown_1464_reduced.root in object /B2L0PbarKpKp_DD/DecayTree;1",nan
6,datadata_2016MagDown_641_reduced.root,Error reading branch p_PX: index -1 is out of bounds for axis 0 with size 0,nan


...


Number of files with errors: 6
List of files with errors:
datadata_2017MagDown_1112_reduced.root
datadata_2017MagDown_1117_reduced.root
datadata_2017MagDown_27_reduced.root
datadata_2017MagDown_452_reduced.root
datadata_2017MagDown_62_reduced.root
datadata_2017MagDown_954_reduced.root

Number of files missing the tree 'B2L0PbarKpKp_DD/DecayTree': 0

Configuration: ('B2L0PbarKpKp', '2017', 'MagUp', 'LL')
Summary of Errors:


,file,error
0,datadata_2017MagUp_1206_reduced.root,Error reading branch Lp_P: index -1 is out of bounds for axis 0 with size 0
1,datadata_2017MagUp_186_reduced.root,Error reading branch Bu_PT: index -1 is out of bounds for axis 0 with size 0
2,datadata_2017MagUp_609_reduced.root,Error reading branch nBu_DTFL0_nDOF: index -1 is out of bounds for axis 0 with size 0
3,datadata_2017MagUp_758_reduced.root,Error reading branch h1_IPCHI2_OWNPV: index -1 is out of bounds for axis 0 with size 0
4,datadata_2017MagUp_833_reduced.root,Error reading branch Lp_PT: index -1 is out of bounds for axis 0 with size 0
5,datadata_2017MagUp_898_reduced.root,Error reading branch h2_PE: index -1 is out of bounds for axis 0 with size 0
6,datadata_2017MagUp_992_reduced.root,Error reading branch Bu_PZ: index -1 is out of bounds for axis 0 with size 0



Number of files with errors: 7
List of files with errors:
datadata_2017MagUp_1206_reduced.root
datadata_2017MagUp_186_reduced.root
datadata_2017MagUp_609_reduced.root
datadata_2017MagUp_758_reduced.root
datadata_2017MagUp_833_reduced.root
datadata_2017MagUp_898_reduced.root
datadata_2017MagUp_992_reduced.root

Number of files missing the tree 'B2L0PbarKpKp_DD/DecayTree': 0

Configuration: ('B2L0PbarKpKp', '2017', 'MagUp', 'DD')
Summary of Errors:


,file,error
0,datadata_2017MagUp_1000_reduced.root,Error reading branch Bu_DTF_status: index -1 is out of bounds for axis 0 with size 0
1,datadata_2017MagUp_1005_reduced.root,Error reading branch Lp_PIDK: index -1 is out of bounds for axis 0 with size 0
2,datadata_2017MagUp_1052_reduced.root,Error reading branch p_PX: while reading TBasket version None as uproot.models.TBasket.Model_TBasket (? bytes) attempting to get bytes 119923:119941 outside expected range 119923:119923 for this Chunk in file /eos/lhcb/user/m/melashri/data/red_RD/B2L0PbarKpKp/2017/MagUp/DD/datadata_2017MagUp_1052_reduced.root
3,datadata_2017MagUp_1092_reduced.root,Error reading branch Bu_MMERR: index -1 is out of bounds for axis 0 with size 0
4,datadata_2017MagUp_1255_reduced.root,Error reading branch h2_P: index -1 is out of bounds for axis 0 with size 0
5,datadata_2017MagUp_498_reduced.root,Error reading branch h1_PIDK: index -1 is out of bounds for axis 0 with size 0
6,datadata_2017MagUp_5_reduced.root,Error reading branch h1_PZ: index -1 is out of bounds for axis 0 with size 0
7,datadata_2017MagUp_639_reduced.root,Error reading branch p_PE: index -1 is out of bounds for axis 0 with size 0
8,datadata_2017MagUp_937_reduced.root,Error reading branch h1_PE: index -1 is out of bounds for axis 0 with size 0



Number of files with errors: 9
List of files with errors:
datadata_2017MagUp_1000_reduced.root
datadata_2017MagUp_1005_reduced.root
datadata_2017MagUp_1052_reduced.root
datadata_2017MagUp_1092_reduced.root
datadata_2017MagUp_1255_reduced.root
datadata_2017MagUp_498_reduced.root
datadata_2017MagUp_5_reduced.root
datadata_2017MagUp_639_reduced.root
datadata_2017MagUp_937_reduced.root

Number of files missing the tree 'B2L0PbarKpKp_DD/DecayTree': 0
